In [1]:
!pip install selenium beautifulsoup4
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


!pip install XlsxWriter


In [2]:
import re
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

# Tạo service
service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service)
# Mở Google
driver.get("https://google.com")

 
# Chờ cho thanh tìm kiếm xuất hiện
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "gLFyf"))
)

# Tương tác với thanh tìm kiếm
input_element = driver.find_element(By.CLASS_NAME, "gLFyf")
input_element.send_keys("nguyenkim" + Keys.ENTER)

# Xử lý CAPTCHA bằng tay (nếu có capcha)
#print("Chờ bạn xử lý CAPTCHA...")
#time.sleep(30)  # Điều chỉnh thời gian tùy ý
#print("Tiếp tục sau khi CAPTCHA đã được xử lý.")

# Chờ kết quả tìm kiếm và nhấp vào liên kết
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Nguyễn Kim - Mua tủ lạnh, máy lạnh, tivi, máy giặt chính hãng"))
)
click_result = driver.find_element(By.PARTIAL_LINK_TEXT, "Nguyễn Kim - Mua tủ lạnh, máy lạnh, tivi, máy giặt chính hãng")
click_result.click()
# Chờ trang sản phẩm tải xong
time.sleep(10)



# # Tạo một dictionary để lưu các sản phẩm theo danh mục
# products_by_category = {}

# # truy cập vào danh mục các sản phẩm
# for link in link1:
#     driver.get(link)
#     WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'nk-product-cate-style-grid')]"))
#     )

#     # Cuộn xuống cuối trang để tải toàn bộ sản phẩm của 1 danh mục
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(3)

#     # Lấy tên danh mục sản phẩm để đặt tên sheet
#     category_name = driver.title.split('|')[0].strip()

#     # Lấy link chi tiết các sản phẩm trong danh mục
#     product_links = []
#     product_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'nk-product-cate-style-grid') and contains(@class, 'nk-product-collection')]")
#     for container in product_containers:
#         try:
#             a_tags = container.find_elements(By.TAG_NAME, 'a')
#             for a_tag in a_tags:
#                 href = a_tag.get_attribute('href')
#                 if href:
#                     product_links.append(href)
#                     print(f"Link thu được: {href}")
#         except Exception as e:
#             print(f"Lỗi khi lấy link từ container sản phẩm: {e}")

#     # Lưu các link sản phẩm vào dictionary với key là tên danh mục
#     products_by_category[category_name] = product_links

# # Hàm loại bỏ ký tự không hợp lệ trong tên sheet 
# # ( vì trong sheet không cho đặt tên bằng các ký tự đặc biệt)
# def clean_sheet_name(sheet_name):
#     return re.sub(r'[\\/:*?"<>|]', '', sheet_name)[:31] # sheet có tối đa 31 ký tự
# #________________________________________________________________________________________________________
# # ------------------------- Lấy thông tin từng sản phẩm --------------------------------------
# sanpham = []
# for linksp in link1:
#     try:
#         driver.get(linksp)
#         time.sleep(4)
#          # Cuộn xuống cuối trang để chắc chắn toàn bộ thông tin sản phẩm đã tải
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(2)

#         # Trích xuất tên sản phẩm
#         try:
#             namesp = driver.find_element(By.CLASS_NAME,"product-title").text
#         except:
#             namesp=''

#         # Trích xuất giá bán 
#         try:
#             price = driver.find_element(By.CLASS_NAME,"final-price").text
#         except:
#             price=''
        
#         # Trích xuất các thông tin khác
#         try:
#              info_elements = driver.find_elements(By.CLASS_NAME, "group")  # Dùng find_elements để lấy nhiều phần tử
#              info = '\n'.join([element.text for element in info_elements])  # Kết hợp tất cả thông tin thành chuỗi
#         except:
#             info = ''
#         # Thêm thông tin vào danh sách 
#         sanpham.append({'NAME': namesp, 'PRICE':price , 'INFORMATION': info})
        
#     except Exception as e:
#         print(f"Error processing {linksp}: {e}")  
       
        
#        # __________________________________________________________________________
# # ---------------------- TẠO DATAFRAME  -------------------------------------------
# # Tạo DataFrame và lưu từng danh mục vào một sheet riêng
# file_name = "products_by_category.xlsx"
# with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
#     for category, links in products_by_category.items():
#         if isinstance(links, list) and links:  # Kiểm tra nếu links là danh sách và không rỗng
#             valid_links = [link for link in links if isinstance(link, str)]
#             if valid_links:
#                 cleaned_name = clean_sheet_name(category)  # Làm sạch tên sheet
#                 if cleaned_name:  # Kiểm tra nếu cleaned_name không rỗng
#                     df = pd.DataFrame(valid_links, columns=['Link Sản Phẩm'])
#                     df.to_excel(writer, sheet_name=cleaned_name, index=False)
#                 else:
#                     print(f"Tên danh mục '{category}' sau khi làm sạch không hợp lệ.")
#             else:
#                 print(f"Danh mục '{category}' không có liên kết hợp lệ.")
#         else:
#             print(f"Danh mục '{category}' không có dữ liệu hoặc không phải là danh sách.")
# print(f"Đã lưu thành công vào {file_name}")


# # Tạo data frame sanpham.xlsx
# sanpham_df =pd.DataFrame(sanpham)
# # Lưu vào file EXcel
# file_name="SP.xlsx"
# sanpham_df.to_excel(file_name,index=False)
# print('ĐÃ lưu thành công các sản phẩm ')

# #___________________________________________________________________________________________________
# # Đóng trình duyệt
# driver.quit()


In [3]:
# Cuộn đến phần tử cần lấy dữ liệu
scroll = driver.find_element(By.CLASS_NAME,'nk-product-cate-homepage')
driver.execute_script("arguments[0].scrollIntoView();", scroll)
time.sleep(3)

# Lấy link các danh mục sản phẩm
# Tạo danh sách rỗng với tên biến là link1
link1 = []
# Biến cl_tags để tìm tất cả phần tử có class name là nl-content
cl_tags = driver.find_elements(By.CLASS_NAME, 'nk-content')
# Tạo vòng lập for với biến tag để truy xuất dữ liệu trong tất cả phần tử cl_tags
for tag in cl_tags:
    try:
        # Biến a_tags để lấy tất cả thẻ <a>
        a_tags = tag.find_elements(By.TAG_NAME, 'a')
        # Vòng lập for để lấy từng thẻ <a>
        for a_tag in a_tags:
            # Biến href được gán giá trị từ thẻ <a> với thuộc tính là href 
            href = a_tag.get_attribute('href')
            if href:
                # Thêm phần tử href vào link1
                link1.append(href)
    # Xử lý ngoại lệ            
    except Exception as e:
        print(f"Đã xảy ra lỗi: {e}")

In [4]:
# Tạo danh sách product_links để lưu link của từng sản phẩm cụ thể
product_links = []
# Tạo vòng lập để duyệt qua từng link
for link in link1:
    # Mở link
    driver.get(link)
    # Vòng lập while để thực hiện lấy tất cả link cụ thể của từng sản phẩm trong từng danh mục đã lưu vào biến link1
    while True:
        try:
            # Chờ để web load dữ liệu
            WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'nk-product-cate-style-grid')]"))
            )
            # Lấy các container sản phẩm dựa trên các div có 2 lớp cụ thể là nk-product-cate-style-grid và nk-product-collectio
            product_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'nk-product-cate-style-grid') and contains(@class, 'nk-product-collection')]")
            # Vòng lập để lấy từng product_containers cụ thể            
            for container in product_containers:
                try:
                    # Tìm tất cả các thẻ <a> trong container
                    a_tags = container.find_elements(By.TAG_NAME, 'a')
                    # Lấy tất cả thẻ <a>
                    for a_tag in a_tags:
                        # Lấy href của tất cả thẻ <a>
                        href = a_tag.get_attribute('href')
                        if href and href not in product_links:  # Kiểm tra để không thêm trùng link
                            # Thêm vào product_links
                            product_links.append(href)
                            # In ra các link từng sản phẩm
                            print(f"Link thu được: {href}") 
                # Xử lý ngoại lệ                    
                except Exception as e:
                    print(f"Lỗi khi lấy link từ container sản phẩm: {e}")
                # Xử lí chuyển page kế tiếp
                try:
                    # Tìm phần tử của thẻ <div>
                    div_element = driver.find_element(By.XPATH, "//div[@class='NkReview_footer_col-3']")
                    # Lấy thẻ a trong thẻ <div>
                    a_element = div_element.find_element(By.TAG_NAME, 'a')
                    # Lấy href của thẻ <a>
                    next_page = a_element.get_attribute('href')
                    # Mở trang tiếp theo
                    driver.get(next_page)
                # Kết thúc vòng lập nếu hết page
                except:
                    continue
        # Xử lý ngoại lệ        
        except:
            print("Lỗi thu thập")
            continue
        break    


Link thu được: https://www.nguyenkim.com/may-lanh-casper-inverter-tc-09is35.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=128074
Link thu được: https://www.nguyenkim.com/may-lanh-panasonic-inverter-cu-cs-pu12zkh-8m.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=127689
Link thu được: https://www.nguyenkim.com/may-lanh-reetech-1-hp-rt9-rc9-ta-bt.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=128121
Link thu được: https://www.nguyenkim.com/may-lanh-panasonic-inverter-cu-cs-xu9zkh-8.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=127668
Link thu được: https://www.nguyenkim.com/may-lanh-daikin-ftky25wmvmv.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=116296
Link thu được: https://www.nguyenkim.com/may-lan

In [5]:
print("Tổng sản phẩm: ",len(product_links))

Tổng sản phẩm:  1174


In [6]:
# Tạo danh sách để thêm thông tin chi tiết sản phẩm
data = []
# Tạo vòng lập để lấy thông tin chi tiết sản phẩm
for op_link in product_links:
    try:
        driver.get(op_link)
        time.sleep(3)
        # Lấy tên sản phẩm
        try:
            div_name = driver.find_element(By.CLASS_NAME, "wrap_name_vote")
            name = div_name.find_element(By.TAG_NAME, 'h1').text
        except:
            name = "Sản phẩm không có tên"
        # Lấy giá thành của sản phẩm
        try:
            div_value_final = driver.find_element(By.CLASS_NAME, "product_info_price_value-final")
            # Lấy giá thành của sản phẩm
            value_final = div_value_final.find_element(By.TAG_NAME, 'span').text
        except:
            value_final = ""
        # Lấy giá thành ban đầu của sản phẩm
        try:
            div_value_real = driver.find_element(By.CLASS_NAME, "price_promotion")
            value_real = div_value_real.find_element(By.CLASS_NAME, "product_info_price_value-real").text
        except:
            value_real = ""
        # Lấy thông tin của sản phẩm
        try:
            div_specification = driver.find_element(By.CLASS_NAME, "productSpecification_table")
            move_by_actions = ActionChains(driver)
            # Chuyển tới trang chứa thông tin chi tiết
            move_by_actions.move_to_element(div_specification).perform()
            time.sleep(2)
            # Lấy toàn bộ mã HTML của phần tử (div_specification) chứa thông tin
            html_content = div_specification.get_attribute('outerHTML')
            print(html_content)
            # Sử dụng BeautifulSoup để phân tích cú pháp HTML
            soup = BeautifulSoup(html_content, 'html.parser')
            # Tìm thẻ <tbody> bằng BeautifulSoup
            t_body = soup.find('tbody')
            # Duyệt qua các thẻ <tr> trong <tbody> để lấy thông tin
            rows = t_body.find_all('tr')
            # Lấy thẻ <tr> thứ 2, 3, 4, 5
            tr_second = rows[1]  # Thẻ <tr> thứ 2
            tr_third = rows[2] # Thẻ <tr> thứ 3
            tr_fourth = rows[3] # Thẻ <tr> thứ 4
            tr_fifth = rows[4] # Thẻ <tr> thứ 5
            # Lấy tiêu đề của thẻ <tr> thứ 2
            title_second = tr_second.find('td', class_='title').text.strip()

            # Kiểm tra nếu tiêu đề không phải là "Màu sắc"
            if title_second != "Màu sắc:":
                # Nếu không phải "Màu sắc", thì tr_third sẽ nhận giá trị của tr_second
                value_third = value_second
                # Thay đổi tr_second để nhận giá trị của hàng tiếp theo
                tr_second = rows[2]
                value_second = tr_second.find('td', class_='value').text.strip()
                
                # Cập nhật các giá trị khác cho tr_fourth và tr_fifth
                tr_fourth = rows[3]
                value_fourth = tr_fourth.find('td', class_='value').text.strip()
                
                tr_fifth = rows[4]
                value_fifth = tr_fifth.find('td', class_='value').text.strip()
            else:
                # Nếu tiêu đề đúng là "Màu sắc", xử lý như bình thường
                value_second = tr_second.find('td', class_='value').text.strip()
                value_third = tr_third.find('td', class_='value').text.strip()
                value_fourth = tr_fourth.find('td', class_='value').text.strip()
                value_fifth = tr_fifth.find('td', class_='value').text.strip()
        except:
            value_second = "" 
            value_third = "" 
            value_fourth = "" 
            value_fifth = ""
        # Thêm thông tin vào danh sách
        data.append({'Tên sản phẩm': name, 'Giá khuyến mãi': value_final, 'Giá ban đầu': value_real, 'Màu sắc': value_second,
                      'Nhà sản xuất': value_third, 'Xuất xứ': value_fourth, 'Năm ra mắt': value_fifth})
    except Exception as e:
        print(f"Lỗi: {e}")
data_df = pd.DataFrame(data)
            

<table class="productSpecification_table"><caption class="productSpecification_title">Thông số kỹ thuật</caption><tbody><tr><td class="title">Model:</td><td class="value"> TC-09IS35 </td></tr><tr><td class="title">Màu sắc:</td><td class="value"> Trắng </td></tr><tr><td class="title">Nhà sản xuất:</td><td class="value"> Casper </td></tr><tr><td class="title">Xuất xứ:</td><td class="value"> Thái Lan </td></tr><tr><td class="title">Năm ra mắt :</td><td class="value"> 2023 </td></tr><tr><td class="title">Thời gian bảo hành:</td><td class="value"> 36 Tháng </td></tr><tr><td class="title">Địa điểm bảo hành:</td><td class="value"> Nguyễn Kim </td></tr><tr><td class="title">Loại máy lạnh:</td><td class="value"> Máy lạnh 1 chiều (chỉ làm lạnh) </td></tr><tr><td class="title">Kiểu dáng:</td><td class="value"> Máy lạnh treo tường </td></tr><tr><td class="title">Công suất:</td><td class="value"> 1 HP </td></tr></tbody></table>
<table class="productSpecification_table"><caption class="productSpecif

In [31]:
# Xử lí dữ liệu nhiễu
data_df = data_df[data_df['Tên sản phẩm'] != 'Sản phẩm không có tên']
# Xóa các hàng có cột "Nhà sản xuất" là rỗng
data_df = data_df[data_df["Nhà sản xuất"] != ""]



In [32]:
data_df

,Tên sản phẩm,Giá khuyến mãi,Giá ban đầu,Màu sắc,Nhà sản xuất,Xuất xứ,Năm ra mắt
0,Máy lạnh Casper Inverter 1 HP TC-09IS35,6.490.000đ,7.490.000đ,Trắng,Casper,Thái Lan,2023
2,Máy lạnh Panasonic Inverter 1.5 HP CU/CS-PU12Z...,13.890.000đ,15.970.000đ,Trắng,Panasonic,Malaysia,2023
4,Máy lạnh Reetech 1 HP RT9/RC9-TA-BT,5.790.000đ,7.190.000đ,Trắng,Reetech,Thái Lan,2023
6,Máy lạnh Panasonic Inverter 1 HP CU/CS-XU9ZKH-8,13.990.000đ,16.090.000đ,Trắng,Panasonic,Malaysia,2023
8,Máy lạnh Daikin Inverter 1 HP FTKY25WMVMV,12.990.000đ,13.990.000đ,Trắng,Daikin,Việt Nam,2022
...,...,...,...,...,...,...,...
1164,Máy sấy tóc Philips BHD308/10 Đen,690.000đ,835.000đ,Đen,Philips,Trung Quốc,2023
1166,Máy sấy tóc Bluestone HDB-1863,699.000đ,730.000đ,Đen,BlueStone,Trung Quốc,2022
1168,Máy sấy tóc Panasonic EH-ND65-K645,789.000đ,,Đen,Panasonic,Thái Lan,2021
1170,Máy sấy tóc Philips BHD300/10,790.000đ,795.000đ,Trắng - Hồng,Philips,Trung Quốc,2016


In [33]:
data_df.to_excel("output.xlsx", index=False)